In [ ]:
#hide
from nbdev.showdoc import show_doc

# 10-fold Cross Validation
> Code for training is the same as Notebook 05.

In [ ]:
from car_speech.fname_processing import load_fnames
from car_speech.pipeline import *

from random import shuffle
from copy import deepcopy
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import layers
from tensorflow.keras import models
from sklearn.model_selection import train_test_split, KFold

## Configuration

In [ ]:
DATASET_TYPE = 'digits' # or 'letters'
label_strings = np.array([str(num) for num in range(0,10)])
EPOCHS = 30

# load classified filenames
filenames = load_fnames('noise_levels/digit_noise_levels/35U.data')
num_samples = len(filenames)
print('number of files:', len(filenames))

# shuffle
shuffle(filenames)

fold_no = 1
acc_per_fold = []

number of files: 1590


## Loop for 10-fold

In [ ]:
beg_ratio = 0
while beg_ratio < 0.9:
    test_beg = int(beg_ratio*num_samples)
    test_end = int((beg_ratio+0.1)*num_samples)
    
    fnames_copy = deepcopy(filenames)
    test_files = fnames_copy[test_beg:test_end]
    del fnames_copy[test_beg:test_end]
    
    val_len = len(test_files)
    val_files = fnames_copy[-val_len:]
    del fnames_copy[-val_len:]
    
    train_files = fnames_copy
    
    beg_ratio += 0.1
    
    train_files = shuffle_data(train_files)
    val_files = shuffle_data(val_files)
    test_files = shuffle_data(test_files)
    

    # Process data using the combined pipeline
    spectrogram_ds = preprocess_dataset(train_files, DATASET_TYPE)
    train_ds = spectrogram_ds
    val_ds = preprocess_dataset(val_files, DATASET_TYPE)
    test_ds = preprocess_dataset(test_files, DATASET_TYPE)
    print("Pipeline Completed")
    
    # batch
    batch_size = 64
    train_ds = train_ds.batch(batch_size)
    val_ds = val_ds.batch(batch_size)
    
    AUTOTUNE = tf.data.experimental.AUTOTUNE
    train_ds = train_ds.cache().prefetch(AUTOTUNE)
    val_ds = val_ds.cache().prefetch(AUTOTUNE)
    
    # model
    for spectrogram, _ in spectrogram_ds.take(1):
        input_shape = spectrogram.shape
#     print('Input shape:', input_shape)
    num_labels = len(label_strings)

    norm_layer = preprocessing.Normalization()
    norm_layer.adapt(spectrogram_ds.map(lambda x, _: x))

    model = models.Sequential([
        layers.Input(shape=input_shape),
        preprocessing.Resizing(32, 32), 
        norm_layer,
        layers.Conv2D(32, 3, activation='relu'),
        layers.Conv2D(64, 3, activation='relu'),
        layers.MaxPooling2D(),
        layers.Dropout(0.25),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_labels),
    ])
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy'],
    )
    
    # Train
    history = model.fit(
        train_ds, 
        validation_data=val_ds,  
        epochs=EPOCHS,
        callbacks=tf.keras.callbacks.EarlyStopping(verbose=1, patience=2),
    )
    
    # Test
    test_audio = []
    test_labels = []

    for audio, label in test_ds:
        test_audio.append(audio.numpy())
        test_labels.append(label.numpy())

    test_audio = np.array(test_audio)
    test_labels = np.array(test_labels)
    
    y_pred = np.argmax(model.predict(test_audio), axis=1)
    y_true = test_labels

    test_acc = sum(y_pred == y_true) / len(y_true)
    print(f'Fold {fold_no} - Test set accuracy: {test_acc:.0%}')
    acc_per_fold.append(round(test_acc, 2))
    fold_no += 1

print(acc_per_fold)
print('mean score:', round(np.mean(acc_per_fold), 2))

Pipeline Completed
Epoch 1/30
20/20 [==============================] - 2s 112ms/step - loss: 2.1765 - accuracy: 0.2115 - val_loss: 1.9110 - val_accuracy: 0.3585
Epoch 2/30
20/20 [==============================] - 1s 69ms/step - loss: 1.6624 - accuracy: 0.4253 - val_loss: 1.4312 - val_accuracy: 0.5094
Epoch 3/30
20/20 [==============================] - 1s 69ms/step - loss: 1.3643 - accuracy: 0.5275 - val_loss: 1.2580 - val_accuracy: 0.5157
Epoch 4/30
20/20 [==============================] - 1s 71ms/step - loss: 1.2516 - accuracy: 0.5684 - val_loss: 1.1717 - val_accuracy: 0.5472
Epoch 5/30
20/20 [==============================] - 1s 70ms/step - loss: 1.0913 - accuracy: 0.6242 - val_loss: 1.0671 - val_accuracy: 0.6352
Epoch 6/30
20/20 [==============================] - 1s 69ms/step - loss: 1.0127 - accuracy: 0.6462 - val_loss: 1.0099 - val_accuracy: 0.6855
Epoch 7/30
20/20 [==============================] - 1s 69ms/step - loss: 0.9365 - accuracy: 0.6659 - val_loss: 1.0487 - val_accuracy: 

20/20 [==============================] - 1s 68ms/step - loss: 0.4581 - accuracy: 0.8373 - val_loss: 0.6753 - val_accuracy: 0.7296
Epoch 19/30
20/20 [==============================] - 1s 70ms/step - loss: 0.4206 - accuracy: 0.8569 - val_loss: 0.6988 - val_accuracy: 0.7233
Epoch 20/30
20/20 [==============================] - 1s 68ms/step - loss: 0.4113 - accuracy: 0.8671 - val_loss: 0.7712 - val_accuracy: 0.7233
Epoch 00020: early stopping
Fold 6 - Test set accuracy: 74%
Pipeline Completed
Epoch 1/30
20/20 [==============================] - 2s 103ms/step - loss: 2.1738 - accuracy: 0.2146 - val_loss: 1.9392 - val_accuracy: 0.2956
Epoch 2/30
20/20 [==============================] - 1s 68ms/step - loss: 1.7313 - accuracy: 0.3750 - val_loss: 1.5685 - val_accuracy: 0.4465
Epoch 3/30
20/20 [==============================] - 1s 68ms/step - loss: 1.4110 - accuracy: 0.4874 - val_loss: 1.3355 - val_accuracy: 0.5472
Epoch 4/30
20/20 [==============================] - 1s 68ms/step - loss: 1.2664 - a

Epoch 5/30
20/20 [==============================] - 1s 69ms/step - loss: 1.0571 - accuracy: 0.6431 - val_loss: 1.2469 - val_accuracy: 0.6038
Epoch 6/30
20/20 [==============================] - 1s 69ms/step - loss: 0.9649 - accuracy: 0.6690 - val_loss: 1.1347 - val_accuracy: 0.6164
Epoch 7/30
20/20 [==============================] - 1s 68ms/step - loss: 0.8670 - accuracy: 0.6942 - val_loss: 1.0736 - val_accuracy: 0.6352
Epoch 8/30
20/20 [==============================] - 1s 69ms/step - loss: 0.7933 - accuracy: 0.7390 - val_loss: 1.0864 - val_accuracy: 0.6415
Epoch 9/30
20/20 [==============================] - 1s 69ms/step - loss: 0.7648 - accuracy: 0.7366 - val_loss: 1.0387 - val_accuracy: 0.6604
Epoch 10/30
20/20 [==============================] - 1s 69ms/step - loss: 0.6991 - accuracy: 0.7531 - val_loss: 1.0760 - val_accuracy: 0.6478
Epoch 11/30
20/20 [==============================] - 1s 68ms/step - loss: 0.6448 - accuracy: 0.7689 - val_loss: 1.0538 - val_accuracy: 0.6541
Epoch 00011

In [ ]:
print("Done")
print(acc_per_fold)
print('mean score:', round(np.mean(acc_per_fold), 2))

Done
[0.75, 0.72, 0.77, 0.73, 0.74, 0.74, 0.76, 0.74, 0.74, 0.68]
mean score: 0.74
